In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:
df_all = pd.concat([train_data, val_data], axis=0)
df_all = pd.concat([df_all, test_data], axis=0)

In [ ]:
from sklearn.utils import resample

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
import numpy as np
from sklearn.utils import resample

 # Update the path as necessary

# Create a binary label for a specific disease (e.g., anxiety)
df_all['has_disease'] = df_all['diseases'].apply(lambda x: 'anxiety' in x)  # Update the disease as needed

# Extract features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df_all['selected_posts'].astype(str))
y = df_all['has_disease']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model on the training set
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Initialize lists to store the metrics
f1_scores = []
sensitivities = []
specificities = []

# Convert X_test and y_test to numpy arrays if not already (depends on your previous operations)
X_test_np = X_test.toarray()  # Convert to numpy array if using sparse matrix from CountVectorizer
y_test_np = np.array(y_test)  # Ensure y_test is a numpy array

n_iterations=100

# Bootstrap sampling and metric calculation
for _ in range(n_iterations):
    # Bootstrap sample the test data indices
    indices = resample(np.arange(len(y_test_np)), replace=True)  # Generate indices for resampling
    X_sample = X_test_np[indices]
    y_sample = y_test_np[indices]
    y_pred = model.predict(X_sample)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_sample, y_pred).ravel()

    # Calculate metrics
    f1 = f1_score(y_sample, y_pred)
    sensitivity = recall_score(y_sample, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    # Store the metrics
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the mean and standard deviation for each metric
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivities)
std_sensitivity = np.std(sensitivities)
mean_specificity = np.mean(specificities)
std_specificity = np.std(specificities)

# Print the results
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")




F1 Score: Mean = 0.412, Std = 0.023
Sensitivity: Mean = 0.321, Std = 0.021
Specificity: Mean = 0.977, Std = 0.002


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
import numpy as np
from scipy.sparse import hstack

# Load your dataset
# df_all = pd.read_csv('path_to_your_dataset.csv')  # Update the path as necessary

# Names of the columns representing GAD-7 items
gad7_columns = ['item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'item7']  # Replace with your actual column names

# Calculate the GAD-7 total score for each user
df_all['gad7_score'] = df_all[gad7_columns].sum(axis=1)

# Create a binary label for anxiety based on GAD-7 score (>=10 indicates GAD)
df_all['has_disease'] = df_all['gad7_score'] >= 10

# Extract features using CountVectorizer for the text data
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_text = vectorizer.fit_transform(df_all['selected_posts'].astype(str))

# Use the GAD-7 total score as an additional feature
X_gad7 = df_all[['gad7_score']].values

# Combine text features and GAD-7 score
X_combined = hstack([X_text, X_gad7])
y = df_all['has_disease']

# Initialize the 5-fold Stratified cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store the metrics across folds
f1_scores = []
sensitivity_scores = []
specificity_scores = []

# Train and evaluate the model using 5-fold cross-validation
for train_index, test_index in kf.split(X_combined, y):
    X_train, X_test = X_combined[train_index], X_combined[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Predict on the test fold
    y_pred = model.predict(X_test)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # Calculate metrics
    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    # Store the metrics for this fold
    f1_scores.append(f1)
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

# Calculate the mean and standard deviation for each metric
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivity_scores)
std_sensitivity = np.std(sensitivity_scores)
mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

# Print the results
print("Cross-validation scores:")
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
import numpy as np
from sklearn.utils import resample
 # Update the path as necessary

# Create a binary label for a specific disease (e.g., anxiety)
df_all['has_disease'] = df_all['diseases'].apply(lambda x: 'anxiety' in x)  # Update the disease as needed

# Extract features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df_all['selected_posts'].astype(str))
y = df_all['has_disease']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model on the training set
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Number of bootstrap samples
n_iterations = 1000

# Initialize lists to store the metrics
f1_scores = []
sensitivities = []
specificities = []

# Convert X_test to numpy array if using sparse matrix from CountVectorizer
X_test_np = X_test.toarray() if hasattr(X_test, "toarray") else X_test

# Bootstrap sampling and metric calculation
for _ in range(n_iterations):
    # Bootstrap sample the test data indices
    indices = resample(np.arange(len(y_test)), replace=True)  # Generate indices for resampling
    X_sample = X_test_np[indices]
    y_sample = y_test.iloc[indices].values  # Use .iloc for correct indexing
    y_pred = model.predict(X_sample)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_sample, y_pred).ravel()

    # Calculate metrics
    f1 = f1_score(y_sample, y_pred)
    sensitivity = recall_score(y_sample, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    # Store the metrics
    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the mean and standard deviation for each metric
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivities)
std_sensitivity = np.std(sensitivities)
mean_specificity = np.mean(specificities)
std_specificity = np.std(specificities)

# Print the results
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")



F1 Score: Mean = 0.409, Std = 0.024
Sensitivity: Mean = 0.319, Std = 0.023
Specificity: Mean = 0.977, Std = 0.002


In [ ]:
gad7_keywords = [
    "nervousness",
    "uncontrol",
    "excess",
    "relaxation",
    "restlessness",
    "irritability",
    "fear"
]


In [ ]:
# save
with open('anxiety-chatgpt-logistic-model.pkl','wb') as f:
    pickle.dump(model,f)

# load
with open('anxiety-chatgpt-logistic-model.pkl', 'rb') as f:
    model_loaded = pickle.load(f)

In [ ]:
a=model_loaded.predict(X_combined.tocsr()[0:10])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from scipy.sparse import hstack

# Assuming df_all is your dataset
# Example to ensure the setup matches expected structur

# Convert lists in 'selected_posts' to single strings
df_all['selected_posts'] = df_all['selected_posts'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Define keywords or phrases indicative of each GAD-7 item
gad7_keywords = {
    "nervousness": ["nervous", "anxious", "on edge"],
    "uncontrol": ["can't stop", "uncontrollable"],
    "excess": ["too much", "excessive", "all the time"],
    "relaxation": ["can't relax", "trouble relaxing"],
    "restlessness": ["restless", "can't sit still"],
    "irritability": ["annoyed", "irritable"],
    "fear": ["afraid", "fearful", "something awful"]
}

# Function to estimate GAD-7 ratings based on selected_posts
def estimate_gad7_scores(post, keywords):
    scores = []
    post_lower = post.lower()
    for key, phrases in keywords.items():
        score = any(phrase in post_lower for phrase in phrases)
        scores.append(int(score))  # Simple binary score for presence of keywords
    return scores

# Apply the function to generate GAD-7 ratings
gad7_scores = [estimate_gad7_scores(post, gad7_keywords) for post in df_all['selected_posts']]
X_gad7 = np.array(gad7_scores)

# Process the text data (selected_posts)
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X_text = vectorizer.fit_transform(df_all['selected_posts'])

# Combine text features with estimated GAD-7 scores
X_combined = hstack([X_text, X_gad7])

# Set up the target variable based on the presence of 'anxiety' in the diseases list
y = df_all['diseases'].apply(lambda diseases: 1 if 'anxiety' in diseases else 0).values

# Initialize the 5-fold Stratified cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics storage
f1_scores = []
sensitivity_scores = []
specificity_scores = []

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Cross-validation process
for train_index, test_index in kf.split(X_combined, y):
    X_train, X_test = X_combined.tocsr()[train_index], X_combined.tocsr()[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Predict on the test fold
    y_pred = model.predict(X_test)

    # Calculate and store metrics
    f1_scores.append(f1_score(y_test, y_pred))
    sensitivity_scores.append(recall_score(y_test, y_pred))
    specificity_scores.append(specificity_score(y_test, y_pred))

# Calculate the mean and standard deviation for each metric
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivity_scores)
std_sensitivity = np.std(sensitivity_scores)
mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

# Print the results
print("Cross-validation scores:")
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")


Cross-validation scores:
F1 Score: Mean = 0.431, Std = 0.022
Sensitivity: Mean = 0.319, Std = 0.019
Specificity: Mean = 0.985, Std = 0.002


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from scipy.sparse import hstack

# Assuming df_all is already defined in your environment
# Make sure to convert lists in 'selected_posts' to single strings
df_all['selected_posts'] = df_all['selected_posts'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Define keywords or phrases indicative of each GAD-7 item
gad7_keywords = {
    "nervousness": ["nervous", "anxious", "on edge"],
    "uncontrol": ["can't stop", "uncontrollable"],
    "excess": ["too much", "excessive", "all the time"],
    "relaxation": ["can't relax", "trouble relaxing"],
    "restlessness": ["restless", "can't sit still"],
    "irritability": ["annoyed", "irritable"],
    "fear": ["afraid", "fearful", "something awful"]
}

# Function to estimate GAD-7 ratings based on selected_posts
def estimate_gad7_scores(post, keywords):
    scores = []
    post_lower = post.lower()
    for key, phrases in keywords.items():
        score = any(phrase in post_lower for phrase in phrases)
        scores.append(int(score))  # Simple binary score for presence of keywords
    return scores

# Apply the function to generate GAD-7 ratings
gad7_scores = [estimate_gad7_scores(post, gad7_keywords) for post in df_all['selected_posts']]
X_gad7 = np.array(gad7_scores)

# Process the text data (selected_posts) using CountVectorizer instead of TfidfVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_text = vectorizer.fit_transform(df_all['selected_posts'])

# Combine text features with estimated GAD-7 scores
X_combined = hstack([X_text, X_gad7])

# Set up the target variable based on the presence of 'anxiety' in the diseases list
y = df_all['diseases'].apply(lambda diseases: 1 if 'anxiety' in diseases else 0).values

# Initialize the 5-fold Stratified cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics storage
f1_scores = []
sensitivity_scores = []
specificity_scores = []

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Cross-validation process
for train_index, test_index in kf.split(X_combined, y):
    X_train, X_test = X_combined.tocsr()[train_index], X_combined.tocsr()[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Predict on the test fold
    y_pred = model.predict(X_test)

    # Calculate and store metrics
    f1_scores.append(f1_score(y_test, y_pred))
    sensitivity_scores.append(recall_score(y_test, y_pred))
    specificity_scores.append(specificity_score(y_test, y_pred))

# Calculate the mean and standard deviation for each metric
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivity_scores)
std_sensitivity = np.std(sensitivity_scores)
mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

# Print the results
print("Cross-validation scores:")
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")


Cross-validation scores:
F1 Score: Mean = 0.409, Std = 0.018
Sensitivity: Mean = 0.326, Std = 0.011
Specificity: Mean = 0.975, Std = 0.003


In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from scipy.sparse import hstack
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Assuming df_all is already defined in your environment
df_all['selected_posts'] = df_all['selected_posts'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

gad7_keywords = {
    "nervousness": ["nervous", "anxious", "on edge"],
    "uncontrol": ["can't stop", "uncontrollable"],
    "excess": ["too much", "excessive", "all the time"],
    "relaxation": ["can't relax", "trouble relaxing"],
    "restlessness": ["restless", "can't sit still"],
    "irritability": ["annoyed", "irritable"],
    "fear": ["afraid", "fearful", "something awful"]
}

# Define enhanced keyword extraction and sentiment analysis
def enhanced_score_estimation(post):
    doc = nlp(post)
    sentiment_score = sentiment_analyzer.polarity_scores(post)['compound']

    scores = []
    for key, phrases in gad7_keywords.items():
        # Consider both occurrence and sentiment
        phrase_score = sum(token.lemma_ in phrases for token in doc)
        scores.append(phrase_score * sentiment_score)  # Weight by sentiment
    return scores

# Apply the function to generate GAD-7 ratings
gad7_scores = [enhanced_score_estimation(post) for post in df_all['selected_posts']]
X_gad7 = np.array(gad7_scores)

# Process the text data
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_text = vectorizer.fit_transform(df_all['selected_posts'])

# Combine features
X_combined = hstack([X_text, X_gad7])
y = (df_all['diseases'].apply(lambda diseases: 1 if 'anxiety' in diseases else 0)).values

# Cross-validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics
f1_scores = []
sensitivity_scores = []
specificity_scores = []

# Cross-validation process
for train_index, test_index in kf.split(X_combined, y):
    X_train, X_test = X_combined[train_index], X_combined[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    f1_scores.append(f1_score(y_test, y_pred))
    sensitivity_scores.append(recall_score(y_test, y_pred))
    specificity_scores.append(specificity_score(y_test, y_pred))

# Output results
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_sensitivity = np.mean(sensitivity_scores)
std_sensitivity = np.std(sensitivity_scores)
mean_specificity = np.mean(specificity_scores)
std_specificity = np.std(specificity_scores)

print("Cross-validation scores:")
print(f"F1 Score: Mean = {mean_f1:.3f}, Std = {std_f1:.3f}")
print(f"Sensitivity: Mean = {mean_sensitivity:.3f}, Std = {std_sensitivity:.3f}")
print(f"Specificity: Mean = {mean_specificity:.3f}, Std = {std_specificity:.3f}")


TypeError: 'coo_matrix' object is not subscriptable